In [127]:

from random import randint
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime

frequency = '6H'   #period of data segmentation in time (hours)
number_zones = 100 #it is automatically changed if reduce = True
reduce = False  #reduce 100 zones to the 10 zones with max activity

In [130]:

data = pd.read_pickle('DR_Data/rentals_inner_cph.pickle')
data = data.sort_values('created_at')
#print(data.shape)
start = data.created_at.iloc[0].floor('H') - datetime.timedelta(hours=10)
end = data.created_at.iloc[-1].ceil('H') -  datetime.timedelta(hours=1)
#print(start,end)
index = pd.date_range(start=start,end=end,freq= frequency)
data.created_at = data.created_at.dt.floor(frequency)



    
    
    
data = data.groupby(['created_at','pickup_zone']).count()
data['Count'] = data['finished_at']
data = data[['Count']].reset_index(1)
print(data.groupby(['pickup_zone']).sum().sort_values('Count',ascending=False).index[:10].values)
x = [randint(0, 99) for p in range(0, 10)]

if reduce:
    number_zones = 10
    zones_with_max_pickups = data.groupby(['pickup_zone']).sum().sort_values('Count',ascending=False).index[:10].values
    data = data[data.pickup_zone.isin(zones_with_max_pickups)]
    values_to_replace = dict(zip(zones_with_max_pickups,range(100,100+number_zones)))
    data = data.replace({"pickup_zone": values_to_replace}) 
    data.pickup_zone = data.pickup_zone - 100

#print(data.groupby(['pickup_zone']).sum())






[52 15 48 25 42 53  4 77 47 30]


In [131]:
def cycle_encoder(data, col, max_val,name):
    data[name + '_sin'] = np.round(np.sin(2 * np.pi * col/max_val),4)
    data[name + '_cos'] = np.round(np.cos(2 * np.pi * col/max_val),4)
    return data

def helper2(time):
    return time.weekday()
    
def helper3(time):
    return time.month

def helper4(time):
    return time.hour


    

def assign_count_to_hub(dt):
    v = np.zeros((number_zones,1))
    v[dt['pickup_zone']] = np.array([dt['Count']]).T
    return v.astype(int).squeeze()
        
df = pd.DataFrame(index=index,columns=['Hubs'])
su=np.zeros((number_zones,1))
for time in index:
    try:
        df.loc[time,'Hubs'] = assign_count_to_hub(data.loc[time])  #if 0 pickups in some interval -> error
    except:
        df.loc[time,'Hubs'] = np.zeros((number_zones,1)).astype(int).squeeze()
    su = su+df.loc[time,'Hubs']
    
df = df.reset_index()
df = cycle_encoder(df, df['index'].apply(helper2), 7,'weekday')
df = cycle_encoder(df, df['index'].apply(helper3), 12,'month')
df = cycle_encoder(df, df['index'].apply(helper4), 24,'hour')
df = df.set_index('index')
print(df)


                                               Hubs  weekday_sin  weekday_cos  \
index                                                                           
2018-03-04 00:00:00  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]      -0.7818       0.6235   
2018-03-04 06:00:00  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]      -0.7818       0.6235   
2018-03-04 12:00:00  [0, 0, 0, 0, 0, 0, 0, 3, 0, 0]      -0.7818       0.6235   
2018-03-04 18:00:00  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]      -0.7818       0.6235   
2018-03-05 00:00:00  [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]       0.0000       1.0000   
...                                             ...          ...          ...   
2019-04-01 12:00:00  [3, 1, 6, 0, 9, 1, 0, 4, 0, 0]       0.0000       1.0000   
2019-04-01 18:00:00  [0, 0, 1, 0, 0, 0, 0, 0, 0, 0]       0.0000       1.0000   
2019-04-02 00:00:00  [0, 0, 1, 0, 1, 0, 0, 0, 0, 0]       0.7818       0.6235   
2019-04-02 06:00:00  [1, 0, 3, 0, 2, 0, 0, 1, 0, 0]       0.7818       0.6235   
2019-04-02 12:00:00  [0, 0, 

In [132]:
weather = pd.read_csv('DR_Data/SingleHour_Weather.csv')
weather['time'] = pd.to_datetime(weather.Day+' '+weather['Time(utc)'])
weather = weather.set_index('time')
weather = weather[['GHI',  'wind_speed_avg' , 'air_temperature','rain_accumulation', 'rain_duration'  ,'rain_intensity']].resample('6H').mean()
weather = weather[(weather.index>=start) & (weather.index<=end)]


In [133]:
print(df.index[0]==weather.index[0])
df = df.join(weather).reset_index()


True


In [134]:
#df.to_pickle('DR_Data/pickup_series_6h.pickle')
#df.to_pickle('DR_Data/pickup_series_10max_zones.pickle')
#df.to_pickle('DR_Data/pickup_series_1_zone.pickle')
#df.to_pickle('DR_Data/pickup_series_6h_weather.pickle')
#df.to_pickle('DR_Data/pickup_series_6h_10random.pickle')